# Simple Assignment (Adapter) Demo
In this we look at another builtin adapter, the Simple Assignment Adapter `enbios/base/adapters_aggregators/builtin/simple_assignment_adapter.py`. This adapter does not any specific calculations. Instead, it allows the user to introduce fixed values, that should come from some external source into the enbios tree calculation. This includes not just the outputs of structural nodes, but also their impact results.
These values can be either in the experiment configuration file or for convenience in a referenced csv file. The values can be specified in such a way, that scenario outputs and result values have consistent and valid units.

In addition, compared to the brightway adapter, this adapter allows to specify multiple outputs for each node, and also nodes that do not specify any output, even tho they have some impacts. 

The adapters dedicated name is `simple-assignment-adapter` tho for nodes, the indicator `assign` can also be used.

We will start with a minimal experiment configuration, that includes one node with a simple assignment 

In [5]:
from enbios import Experiment

# let's keep this node separate, so we can manipulate it easier later
simple_assignment_node = {
    "name": "simple_ass",
    "adapter": "assign",
    # the config of a assign node needs either: default_impacts or scenario_data.impacts
    "config": {
        "output_units": ["l"],
        "default_impacts": {
            "co2": {"unit": "kg", "magnitude": 10}
        }}
}

experiment_data = {
    "adapters": [{
        "adapter_name": "simple-assignment-adapter",
        # by default this adapter does no specific config
        "methods": {
            "co2": "kg"
        }
    }],
    "hierarchy": {
        "name": "root",
        "aggregator": "sum",
        "children": [
            simple_assignment_node
        ]
    }
}

In [6]:
experiment = Experiment(experiment_data)
experiment.run()

2024-03-20 12:56:10,449 - ............demos.enbios.base - INFO - Running scenario 'default scenario'


{'default scenario': {'name': 'root',
  'results': {'co2': {'unit': 'kg', 'magnitude': 10.0}},
  'output': [],
  'children': [{'name': 'simple_ass',
    'results': {'co2': {'unit': 'kg', 'magnitude': 10.0}},
    'output': []}]}}

In [8]:
experiment_data["scenarios"] = [{
    "name": "scenario1",
    "nodes": {
        "simple_ass": {
            "outputs": [{
                "unit": "l", "magnitude": 100
            }],
            "impacts": {
                "co2": {"unit": "pf", "magnitude": 20}
            }
        }
    }
}
]

experiment = Experiment(experiment_data)
experiment.run()

2024-03-20 12:56:22,321 - ............demos.enbios.base - INFO - Running scenario 'scenario1'


{'scenario1': {'name': 'root',
  'results': {'co2': {'unit': 'pf', 'magnitude': 20.0}},
  'output': [{'unit': 'liter', 'magnitude': 100.0}],
  'children': [{'name': 'simple_ass',
    'results': {'co2': {'unit': 'pf', 'magnitude': 20.0}},
    'output': [{'unit': 'l', 'magnitude': 100.0}]}]}}

In [ ]:
experiment.adapters[0].nodes